In [35]:
##Some code to run at the beginning of the file, to be able to show images in the notebook
##Don't worry about this cell

#Print the plots in this screen
%matplotlib inline 

#Be able to plot images saved in the hard drive
from IPython.display import Image 

#Make the notebook wider
from IPython.core.display import display, HTML 
display(HTML("<style>.container { width:90% !important; }</style>"))

import seaborn as sns
import pylab as plt
import pandas as pd
import numpy as np
import scipy.stats

import statsmodels.formula.api as smf
from linearmodels import PanelOLS,PooledOLS,RandomEffects

## Plan
- class6a_stats, look at summary tables
- Review multilevel models
- Enter slightly into panel models

## 6.2 Hierarchical models / Mixed  models / Multilevel models
- A normal model (with 1 or more independent variables) is a fixed effects model
    - Random effects are:
    - Factors likely to introduce systematic variation
    - Different experimental units (groups) have different intercepts of slopes.
    
- You can control for in what is called a random effects model.
    - **Random intercept model**: For instance, we want to measure the relationship between unemployment and productivity for different regions in many countries. Each country may have their own "structural" unemployment, we can get rid of this variability by using country as the random effects. --> (1|country)


    - **Random intercept and slopes model**: Another example, we measure the relationship between two variables at different years in different countries. We don't really care about how they change in time, we only care about how one variable affect the other. We can use time as the random effect. --> (year|country)
        
    - You can have multiple random effects. In that case the random effects can be:
        - Nested: E.g. region within country --> (1|country/region)
        - Crossed: E.g. sector & country --> (1|sector) + (1|country)

    - The design can be balanced (same number of observation per group) or unbalanced (different number). Unbalanced designs require special techniques, usually handled by the package.

_Here paradox_        



## 6.3 Longitudinal vs cross-sectional
- Having time information is great:
    - Allow us to test for Granger causality (x happens before y, thus x could be a cause of y)
    - Allow us to increase the number of observations
- Having time information is terrible:
    - The observations are not independent anymore, we need to use hierarchical models.
        - Pooled OLS (not a hierarchical model, usually not appropriate)
        - **Between model** (also called random effects model)
            - Looks at the effects of changes of x in y in time within and between individuals at the same time.
            - If you have an omitted variables (and it is correlated with one of your independent variables) you'll have a problem.
            - If you don't have an omitted variable then it's great.
        - **Within model** (also called fixed effects model)
            - Looks at the effects of changes of x in y in time only within individuals.
            - The fixed effects are the subject-specific means.
            - If the subjects don't change much in time you have a problem.
            - You cannot include time-invariant values (such as gender).
        - Mundlack. 
            - Looks at the effects of changes of x in y in time within and between individuals differenciating between the two.
            - It's always great.
    - If your aim is to predict or to look at complex time relationships that you cannot control for then you need to use other type of models such as AR, I, MA (or a combination).
    
Note: Panel means you have repeated measurements for each individual (or whatever your unit of analysis is).

In [3]:
from statsmodels.datasets import grunfeld
data = grunfeld.load_pandas().data
data.year = data.year.astype(np.int64)
data = data.set_index(['firm','year'])
data.head()

invest   value  capital
firm              year                         
b'General Motors' 1935   317.6  3078.5      2.8
                  1936   391.8  4661.7     52.6
                  1937   410.6  5387.1    156.9
                  1938   257.7  2792.2    209.2
                  1939   330.8  4313.2    203.4

In [ ]:
from linearmodels import PanelOLS,PooledOLS,RandomEffects

## Pooled OLS
- It is a normal regression

In [4]:
#
mod_pooled = PooledOLS.from_formula('invest ~ 1 + value + capital', data)
res_pooled = mod_pooled.fit(cov_type='clustered', cluster_entity=True)
print(res_pooled.summary)


                          PooledOLS Estimation Summary                          
Dep. Variable:                 invest   R-squared:                        0.8179
Estimator:                  PooledOLS   R-squared (Between):              0.8426
No. Observations:                 220   R-squared (Within):               0.7357
Date:                Tue, Jan 30 2018   R-squared (Overall):              0.8179
Time:                        00:13:10   Log-likelihood                   -1301.3
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      487.28
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,217)
Min Obs:                       20.000                                           
Max Obs:                       20.000   F-statistic (robust):             52.506
                            

/opt/anaconda/anaconda3/lib/python3.5/site-packages/linearmodels/panel/model.py:603: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = lstsq(wx, wy)[0]


#https://www.statalist.org/forums/forum/general-stata-discussion/general/1314121-r-squared-within-between-overall
 - Between R2 is "How much of the variance between seperate panel units does my model account for"
 - Within R2 is "How much of the variance within the panel units does my model account for"
 - Overall R2 is a weighted average of these two.

## Between regression
- Idea: Looks at the average of all year points
- When to use it:
    - If we believe there is no omitted variable bias
    - If we are interested in time-invariant variables (gender, sector, country...)
- Simpsons paradox goes here!

In [8]:
mod_between = RandomEffects.from_formula('invest ~ 1 + value + capital + EntityEffects', data)
res_between = mod_random.fit(cov_type='clustered', cluster_entity=True)
print(res_between.variance_decomposition)
print(res_between.theta.head()) #1 = pooled
print(res_between.summary)

Effects                   6201.934625
Residual                  2530.041846
Percent due to Effects       0.710256
Name: Variance Decomposition, dtype: float64
                         theta
entity                        
b'General Motors'     0.858616
b'US Steel'           0.858616
b'General Electric'   0.858616
b'Chrysler'           0.858616
b'Atlantic Refining'  0.858616
                        RandomEffects Estimation Summary                        
Dep. Variable:                 invest   R-squared:                        0.7700
Estimator:              RandomEffects   R-squared (Between):              0.8204
No. Observations:                 220   R-squared (Within):               0.7666
Date:                Tue, Jan 30 2018   R-squared (Overall):              0.8080
Time:                        00:16:06   Log-likelihood                   -1172.9
Cov. Estimator:             Clustered                                           
                                        F-statistic:     

/opt/anaconda/anaconda3/lib/python3.5/site-packages/linearmodels/panel/model.py:1731: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = np.linalg.lstsq(x, y)[0]
/opt/anaconda/anaconda3/lib/python3.5/site-packages/linearmodels/panel/model.py:1736: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = np.linalg.lstsq(wxbar, wybar)[0]
/opt/anaconda/anaconda3/lib/python3.5/site-packages/linearmodels/panel/model.py:1770: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)

## Within regression
- Looks only within the group
- No time invariant things (like gender)
- When to use it:
    - If we suspect of omitted variable bias
    - If we are not interested in time-invariant variables (gender, sector, country...)
- Only feasible if enough variability within subjects (usually many years)


In [17]:
mod_within = PanelOLS.from_formula('invest ~ 1 + value + capital + EntityEffects', data)
res_within = mod_within.fit(cov_type='clustered', cluster_entity=True)
print(res_within.variance_decomposition)
print(res_within.summary)

Effects                   6133.752486
Residual                  2380.539374
Percent due to Effects       0.720407
Name: Variance Decomposition, dtype: float64
                          PanelOLS Estimation Summary                           
Dep. Variable:                 invest   R-squared:                        0.7667
Estimator:                   PanelOLS   R-squared (Between):              0.8193
No. Observations:                 220   R-squared (Within):               0.7667
Date:                Tue, Jan 30 2018   R-squared (Overall):              0.8071
Time:                        00:18:06   Log-likelihood                   -1167.4
Cov. Estimator:             Clustered                                           
                                        F-statistic:                      340.08
Entities:                          11   P-value                           0.0000
Avg Obs:                       20.000   Distribution:                   F(2,207)
Min Obs:                       

/opt/anaconda/anaconda3/lib/python3.5/site-packages/linearmodels/panel/model.py:1164: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  params = np.linalg.lstsq(x, y)[0]
/opt/anaconda/anaconda3/lib/python3.5/site-packages/linearmodels/panel/model.py:1223: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  weps_pooled = wy - wx @ np.linalg.lstsq(wx, wy)[0]


## Compare models
- Pooled always maximize overall r2 (this is not necessarily good)

In [19]:
from linearmodels.panel import compare
compare({'Between':res_between,'Within':res_within,'Pooled':res_pooled})

,Between,Pooled,Within
Dep. Variable,invest,invest,invest
Estimator,RandomEffects,PooledOLS,PanelOLS
No. Observations,220,220,220
Cov. Est.,Clustered,Clustered,Clustered
R-squared,0.7700,0.8179,0.7667
R-Squared (Within),0.7666,0.7357,0.7667
R-Squared (Between),0.8204,0.8426,0.8193
R-Squared (Overall),0.8080,0.8179,0.8071
F-statistic,363.21,487.28,340.08
P-value (F-stat),0.0000,0.0000,0.0000


In [ ]:
#https://bashtage.github.io/linearmodels/doc/panel/examples/examples.html

In [20]:
data.head()

invest   value  capital
firm              year                         
b'General Motors' 1935   317.6  3078.5      2.8
                  1936   391.8  4661.7     52.6
                  1937   410.6  5387.1    156.9
                  1938   257.7  2792.2    209.2
                  1939   330.8  4313.2    203.4

In [23]:
import rpy2
%load_ext rpy2.ipython
%R require("plm")

The rpy2.ipython extension is already loaded. To reload it, use:
  %reload_ext rpy2.ipython


array([1], dtype=int32)

In [32]:
noindex = data.reset_index()
noindex.head()

,firm,year,invest,value,capital
0,b'General Motors',1935,317.6,3078.5,2.8
1,b'General Motors',1936,391.8,4661.7,52.6
2,b'General Motors',1937,410.6,5387.1,156.9
3,b'General Motors',1938,257.7,2792.2,209.2
4,b'General Motors',1939,330.8,4313.2,203.4


In [31]:
%%R -i noindex

#Random effects
model_m_fR <- plm(invest~value+capital,
           data=noindex,
          index=c("firm","year"),
          model="within") #between for between models
print(summary(model_m_fR))


Oneway (individual) effect Within Model

Call:
plm(formula = invest ~ value + capital, data = noindex, model = "within", 
    index = c("firm", "year"))

Balanced Panel: n=11, T=20, N=220

Residuals :
      Min.    1st Qu.     Median    3rd Qu.       Max. 
-184.00792  -15.66024    0.27161   16.41421  250.75337 

Coefficients :
        Estimate Std. Error t-value  Pr(>|t|)    
value    0.11013    0.01130  9.7461 < 2.2e-16 ***
capital  0.31003    0.01654 18.7439 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Total Sum of Squares:    2244500
Residual Sum of Squares: 523720
R-Squared:      0.76667
Adj. R-Squared: 0.75314
F-statistic: 340.079 on 2 and 207 DF, p-value: < 2.22e-16
